In [1]:
# ============================================
# [4] 矩阵分解推荐系统（Matrix Factorization）
# ============================================
# 学习顺序：第4个 - 深入学习现代推荐系统的核心算法
#
# 本notebook涵盖：
# - 基线模型（Baseline Model）
# - 矩阵分解（Matrix Factorization）
# - 不同核函数（Linear, Sigmoid, RBF）
# - 在线学习（Online Learning）
# - Scikit-learn兼容性
#
# 为什么学这个？
# - ✅ 理解现代推荐系统的核心算法
# - ✅ 学习如何处理大规模稀疏数据
# - ✅ 掌握模型训练和调优方法

# ============================================
# 导入必要的库和设置
# ============================================
# 确保导入项目本地的 matrix_factorization 模块
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 数据处理库
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100  # 设置pandas显示的最大行数

# 推荐系统模型
from matrix_factorization import BaselineModel, KernelMF, train_update_test_split
from sklearn.metrics import mean_squared_error  # 用于计算均方误差
from sklearn.model_selection import train_test_split  # 用于划分训练集和测试集

# 其他工具
import os
import random
import sys

# 自动重载导入的代码（开发时很有用，修改代码后自动重新加载）
%load_ext autoreload
%autoreload 2

# 设置Jupyter notebook显示所有输出（不仅仅是最后一个表达式的结果）
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
    
# 设置随机种子，确保结果可复现
rand_seed = 2
np.random.seed(rand_seed)
random.seed(rand_seed)

# 加载数据

本节将加载MovieLens 100K数据集，这是推荐系统研究中最常用的数据集之一。

**MovieLens数据集来源：https://grouplens.org/datasets/movielens/**

MovieLens 100K数据集包含：
- 943个用户对1682部电影的100,000条评分
- 评分范围：1-5星
- 每个用户至少评分了20部电影

In [2]:
# ============================================
# 加载和准备数据
# ============================================
cols = ['user_id', 'item_id', 'rating', 'timestamp']
# 加载MovieLens 100K数据集
# u.data文件格式：user_id \t item_id \t rating \t timestamp
movie_data = pd.read_csv('../data/ml-100k/u.data', names=cols, sep='\t', usecols=[0, 1, 2], engine='python')

# 分离特征和标签
X = movie_data[['user_id', 'item_id']]  # 特征：用户ID和物品ID
y = movie_data['rating']  # 标签：评分

# 划分训练集和测试集（80%训练，20%测试）
# 用于评估模型在标准场景下的性能
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 准备在线学习的数据划分
# 这个划分用于模拟新用户加入的场景：
# - X_train_initial: 初始训练集（老用户的数据）
# - X_train_update: 新用户的训练数据（用于更新模型）
# - X_test_update: 新用户的测试数据（用于评估模型对新用户的预测能力）
# frac_new_users=0.2 表示20%的用户是新用户
X_train_initial, y_train_initial, X_train_update, y_train_update, X_test_update, y_test_update = train_update_test_split(movie_data, frac_new_users=0.2)

# 查看数据的前10行，了解数据格式
# 输出包含：user_id（用户ID）、item_id（物品ID）、rating（评分1-5）
movie_data.head(10)

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
5,298,474,4
6,115,265,2
7,253,465,5
8,305,451,3
9,6,86,3


# 简单基线模型：全局平均评分

这是最简单的推荐模型，将所有预测都设为训练集的全局平均评分。
这个模型作为基线（baseline），用于对比其他模型的性能提升。

**说明：** 这个模型类似于只使用全局标准差，不考虑用户和物品的个性化特征。

In [3]:
# ============================================
# 全局平均模型：最简单的基线
# ============================================
# 计算训练集的全局平均评分
global_mean = y_train.mean()

# 对所有测试样本预测为全局平均
pred = [global_mean for _ in range(y_test.shape[0])]

# 计算均方误差（MSE）和均方根误差（RMSE）
mse = mean_squared_error(y_test, pred)
rmse = mse ** 0.5

# 输出结果
# RMSE值越小越好，表示预测误差越小
# 这个值作为基线，后续模型应该显著优于这个值
print(f'\nTest RMSE: {rmse:4f}')


Test RMSE: 1.120652


# 基线模型（带偏差）

这个模型在全局平均的基础上，加入了用户偏差（user bias）和物品偏差（item bias）。
模型公式：`r_ui = μ + bias_u + bias_i`
- μ: 全局平均评分
- bias_u: 用户偏差（用户评分倾向，如有些用户习惯给高分，有些给低分）
- bias_i: 物品偏差（物品质量，如好电影平均分高，差电影平均分低）

## 随机梯度下降（SGD）方法

使用随机梯度下降算法来优化用户偏差和物品偏差参数。
SGD通过迭代更新参数，逐步降低训练误差。

In [4]:
%%time
# %%time 魔法命令会显示代码执行时间

# ============================================
# 使用SGD训练基线模型
# ============================================
# 创建基线模型，参数说明：
# - method='sgd': 使用随机梯度下降优化
# - n_epochs=20: 训练20个epoch（遍历整个数据集20次）
# - reg=0.005: L2正则化系数，防止过拟合
# - lr=0.01: 学习率，控制参数更新步长
# - verbose=1: 显示训练过程
baseline_model = BaselineModel(method='sgd', n_epochs=20, reg=0.005, lr=0.01, verbose=1)
baseline_model.fit(X_train, y_train)

# 在测试集上预测
pred = baseline_model.predict(X_test)

# 计算测试集RMSE
mse = mean_squared_error(y_test, pred)
rmse = mse ** 0.5

# 输出结果
# 训练过程会显示每个epoch的训练RMSE，应该逐渐下降
# 测试RMSE应该比全局平均模型（~1.12）显著降低
print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 0.9684318630731157
Epoch  2 / 20  -  train_rmse: 0.9451360315279567
Epoch  3 / 20  -  train_rmse: 0.935107563197747
Epoch  4 / 20  -  train_rmse: 0.9295305574576824
Epoch  5 / 20  -  train_rmse: 0.9258104991898358
Epoch  6 / 20  -  train_rmse: 0.9235683529962192
Epoch  7 / 20  -  train_rmse: 0.9218860038891832
Epoch  8 / 20  -  train_rmse: 0.9206818852189994
Epoch  9 / 20  -  train_rmse: 0.919665235611308
Epoch  10 / 20  -  train_rmse: 0.9189015285813185
Epoch  11 / 20  -  train_rmse: 0.9183718496310471
Epoch  12 / 20  -  train_rmse: 0.917953662778815
Epoch  13 / 20  -  train_rmse: 0.9175332876132267
Epoch  14 / 20  -  train_rmse: 0.9171728567850587
Epoch  15 / 20  -  train_rmse: 0.9169112157648677
Epoch  16 / 20  -  train_rmse: 0.9167476199111908
Epoch  17 / 20  -  train_rmse: 0.9165874217710496
Epoch  18 / 20  -  train_rmse: 0.9162830310221506
Epoch  19 / 20  -  train_rmse: 0.9162539313422113
Epoch  20 / 20  -  train_rmse: 0.9160901402895595

Test RMSE: 

/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 144.  700.  918. ... 1073.  575.  185.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)


In [5]:
# ============================================
# 为用户生成推荐
# ============================================
# 为用户ID=200生成推荐列表
# 默认返回10个推荐物品，按预测评分从高到低排序
# 输出包含：user_id（用户ID）、item_id（物品ID）、rating_pred（预测评分）
baseline_model.recommend(user=200)

,user_id,item_id,rating_pred
388,200,408,5.0
378,200,318,5.0
212,200,169,5.0
790,200,114,5.0
281,200,483,5.0
338,200,64,5.0
726,200,513,5.0
188,200,178,5.0
988,200,1449,5.0
34,200,50,5.0


## 交替最小二乘法（ALS）方法

使用交替最小二乘法来优化用户偏差和物品偏差参数。
ALS方法交替固定一个参数集，优化另一个参数集，通常比SGD更快收敛。

In [6]:
%%time

baseline_model = BaselineModel(method='als', n_epochs = 20, reg = 0.5, verbose=1)
baseline_model.fit(X_train, y_train)

pred = baseline_model.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = mse ** 0.5

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 0.9312489364350157
Epoch  2 / 20  -  train_rmse: 0.9144875214764501
Epoch  3 / 20  -  train_rmse: 0.9134856911195807
Epoch  4 / 20  -  train_rmse: 0.9133800448918423
Epoch  5 / 20  -  train_rmse: 0.9133615794862777
Epoch  6 / 20  -  train_rmse: 0.9133565857003941
Epoch  7 / 20  -  train_rmse: 0.9133544601244424
Epoch  8 / 20  -  train_rmse: 0.9133531004630441
Epoch  9 / 20  -  train_rmse: 0.9133519902067218
Epoch  10 / 20  -  train_rmse: 0.9133509792033206
Epoch  11 / 20  -  train_rmse: 0.9133500175542733
Epoch  12 / 20  -  train_rmse: 0.9133490869495551
Epoch  13 / 20  -  train_rmse: 0.9133481801287349
Epoch  14 / 20  -  train_rmse: 0.9133472939684136
Epoch  15 / 20  -  train_rmse: 0.9133464269599311
Epoch  16 / 20  -  train_rmse: 0.9133455782426871
Epoch  17 / 20  -  train_rmse: 0.9133447472230197
Epoch  18 / 20  -  train_rmse: 0.9133439334215674
Epoch  19 / 20  -  train_rmse: 0.9133431364114416
Epoch  20 / 20  -  train_rmse: 0.9133423557930989

Test RMS

/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 645. 1111.  518. ...  910.  863.   76.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)


## 在线学习：更新新用户

在实际应用中，新用户会不断加入系统。我们需要在不重新训练整个模型的情况下，快速为新用户生成推荐。
这展示了模型的在线更新能力。

In [7]:
# ============================================
# 初始训练：只在老用户数据上训练
# ============================================
# 使用初始训练集（只包含老用户）训练模型
# 这模拟了系统初始状态，只有部分用户数据
baseline_model = BaselineModel(method='sgd', n_epochs=20, lr=0.01, reg=0.05, verbose=1)
baseline_model.fit(X_train_initial, y_train_initial)

# 训练输出显示：
# - 每个epoch的训练RMSE，应该逐渐下降
# - 这个模型只学习了老用户的偏好，对新用户还没有任何信息

Epoch  1 / 20  -  train_rmse: 0.9648624129747326
Epoch  2 / 20  -  train_rmse: 0.9429379808431446
Epoch  3 / 20  -  train_rmse: 0.9331740875462184
Epoch  4 / 20  -  train_rmse: 0.9278556611679618
Epoch  5 / 20  -  train_rmse: 0.9248599588769798
Epoch  6 / 20  -  train_rmse: 0.9228298052633704
Epoch  7 / 20  -  train_rmse: 0.9210614515389461
Epoch  8 / 20  -  train_rmse: 0.9199544830458471
Epoch  9 / 20  -  train_rmse: 0.9188122639616381
Epoch  10 / 20  -  train_rmse: 0.9181089888433811
Epoch  11 / 20  -  train_rmse: 0.9175439727462659
Epoch  12 / 20  -  train_rmse: 0.9171332381866416
Epoch  13 / 20  -  train_rmse: 0.9168555075181691
Epoch  14 / 20  -  train_rmse: 0.916581718012277
Epoch  15 / 20  -  train_rmse: 0.9163901101854591
Epoch  16 / 20  -  train_rmse: 0.9160379230433262
Epoch  17 / 20  -  train_rmse: 0.9159162174011831
Epoch  18 / 20  -  train_rmse: 0.9157069637941846
Epoch  19 / 20  -  train_rmse: 0.9154795245668191
Epoch  20 / 20  -  train_rmse: 0.9155308895961631


BaselineModel(n_epochs=20, reg=0.05)

In [8]:
%%time
# ============================================
# 在线更新：为新用户更新模型
# ============================================
# 使用新用户的训练数据更新模型
# update_users方法只更新新用户的参数，不改变已有用户的参数
# 参数说明：
# - n_epochs=20: 更新20个epoch
# - lr=0.001: 较小的学习率，避免破坏已有模型
# - verbose=1: 显示更新过程
baseline_model.update_users(X_train_update, y_train_update, n_epochs=20, lr=0.001, verbose=1)

# 在新用户的测试集上预测
pred = baseline_model.predict(X_test_update)

# 计算新用户测试集的RMSE
mse = mean_squared_error(y_test_update, pred)
rmse = mse ** 0.5

# 输出结果
# 这个RMSE衡量模型对新用户的预测能力
# 由于新用户数据较少，RMSE可能略高于老用户（~0.95 vs ~0.93）
print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 1.0194230954163581
Epoch  2 / 20  -  train_rmse: 1.0027468873839778
Epoch  3 / 20  -  train_rmse: 0.9902703373865163
Epoch  4 / 20  -  train_rmse: 0.9808051388389242
Epoch  5 / 20  -  train_rmse: 0.9735190830574357
Epoch  6 / 20  -  train_rmse: 0.9677633285272387
Epoch  7 / 20  -  train_rmse: 0.9631487397750839
Epoch  8 / 20  -  train_rmse: 0.9593811436693908
Epoch  9 / 20  -  train_rmse: 0.9562492530582372
Epoch  10 / 20  -  train_rmse: 0.9536213478303944
Epoch  11 / 20  -  train_rmse: 0.9513772805124044
Epoch  12 / 20  -  train_rmse: 0.9494415400643412
Epoch  13 / 20  -  train_rmse: 0.9477453554625851
Epoch  14 / 20  -  train_rmse: 0.9462394202103148
Epoch  15 / 20  -  train_rmse: 0.9449061849926357
Epoch  16 / 20  -  train_rmse: 0.9437145260173291
Epoch  17 / 20  -  train_rmse: 0.9426422640259307
Epoch  18 / 20  -  train_rmse: 0.9416658614644714
Epoch  19 / 20  -  train_rmse: 0.9407799344915904
Epoch  20 / 20  -  train_rmse: 0.9399683965250846

Test RMS

/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[198. 354. 862. ... 146.  23. 273.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)


# 矩阵分解（Matrix Factorization）

矩阵分解是更强大的推荐系统方法，通过将用户-物品评分矩阵分解为两个低维矩阵来学习潜在特征。
模型公式：`r_ui = μ + bias_u + bias_i + P_u · Q_i^T`
- P: 用户特征矩阵（n_users × n_factors）
- Q: 物品特征矩阵（n_items × n_factors）
- n_factors: 潜在因子数量（如100）

## 线性核（Linear Kernel）

线性核是最简单的核函数，直接使用用户特征和物品特征的点积。
公式：`K(u,i) = P_u · Q_i^T`
这是标准的矩阵分解方法。

In [9]:
%%time
# ============================================
# 使用线性核的矩阵分解
# ============================================
# 创建核矩阵分解模型，参数说明：
# - n_epochs=20: 训练20个epoch
# - n_factors=100: 潜在因子数量（用户和物品特征向量的维度）
# - kernel='linear': 使用线性核（默认）
# - lr=0.001: 学习率
# - reg=0.005: L2正则化系数
# - verbose=1: 显示训练过程
matrix_fact = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact.fit(X_train, y_train)

# 在测试集上预测
pred = matrix_fact.predict(X_test)

# 计算测试集RMSE
mse = mean_squared_error(y_test, pred)
rmse = mse ** 0.5

# 输出结果
# 矩阵分解模型应该比基线模型性能更好（RMSE更低）
# 训练过程显示每个epoch的RMSE逐渐下降
# 测试RMSE通常在0.95左右，比基线模型（~0.93）略高，但能捕获更复杂的模式
print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 1.0801503618413235
Epoch  2 / 20  -  train_rmse: 1.0473724844438044
Epoch  3 / 20  -  train_rmse: 1.0244909801278825
Epoch  4 / 20  -  train_rmse: 1.007509319072069
Epoch  5 / 20  -  train_rmse: 0.9942542189551763
Epoch  6 / 20  -  train_rmse: 0.9835041008959404
Epoch  7 / 20  -  train_rmse: 0.9745160533435606
Epoch  8 / 20  -  train_rmse: 0.9668131547626996
Epoch  9 / 20  -  train_rmse: 0.9600692186783407
Epoch  10 / 20  -  train_rmse: 0.9540554230192841
Epoch  11 / 20  -  train_rmse: 0.9486196100470438
Epoch  12 / 20  -  train_rmse: 0.9436402044621872
Epoch  13 / 20  -  train_rmse: 0.9390311700481341
Epoch  14 / 20  -  train_rmse: 0.9347266814558365
Epoch  15 / 20  -  train_rmse: 0.9306751919870581
Epoch  16 / 20  -  train_rmse: 0.9268343128731258
Epoch  17 / 20  -  train_rmse: 0.923172186010546
Epoch  18 / 20  -  train_rmse: 0.9196601633411041
Epoch  19 / 20  -  train_rmse: 0.9162767100481671
Epoch  20 / 20  -  train_rmse: 0.9130034045109853

Test RMSE:

/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 616.  581.  212. ...  796. 1194.  152.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)


## 为用户生成推荐列表

展示如何使用训练好的模型为用户生成个性化推荐。

In [10]:
# ============================================
# 为用户生成推荐
# ============================================
user = 200  # 目标用户ID

# 获取用户已经评分过的物品（这些物品不应该出现在推荐列表中）
items_known = X_train.query('user_id == @user')['item_id']

# 生成推荐
# 默认返回10个推荐物品，按预测评分从高到低排序
# 输出包含：user_id（用户ID）、item_id（物品ID）、rating_pred（预测评分）
# 这些物品是用户还没有评分过，但模型预测用户可能会喜欢的
matrix_fact.recommend(user=user, items_known=items_known)

,user_id,item_id,rating_pred
37,200,64,5.000000
242,200,357,4.953133
11,200,127,4.918218
61,200,272,4.905290
710,200,479,4.838466
395,200,480,4.837849
275,200,12,4.818254
655,200,427,4.809626
55,200,511,4.807016
144,200,285,4.798449


## 矩阵分解的在线学习：更新新用户

展示矩阵分解模型如何在线更新，为新用户快速学习偏好。

In [11]:
# ============================================
# 初始训练：只在老用户数据上训练矩阵分解模型
# ============================================
# 使用初始训练集（只包含老用户）训练模型
matrix_fact = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact.fit(X_train_initial, y_train_initial)

# 训练输出显示：
# - 每个epoch的训练RMSE逐渐下降
# - 模型学习了老用户的潜在特征和物品的潜在特征
# - 此时模型对新用户还没有任何信息

Epoch  1 / 20  -  train_rmse: 1.0705596652313616
Epoch  2 / 20  -  train_rmse: 1.0382290087153954
Epoch  3 / 20  -  train_rmse: 1.0162000748213786
Epoch  4 / 20  -  train_rmse: 0.9999091788274755
Epoch  5 / 20  -  train_rmse: 0.9872188512703987
Epoch  6 / 20  -  train_rmse: 0.9769174616799939
Epoch  7 / 20  -  train_rmse: 0.9683025119201346
Epoch  8 / 20  -  train_rmse: 0.9608947540619545
Epoch  9 / 20  -  train_rmse: 0.9543933548467968
Epoch  10 / 20  -  train_rmse: 0.9485807223161451
Epoch  11 / 20  -  train_rmse: 0.943307647097417
Epoch  12 / 20  -  train_rmse: 0.9384642445036238
Epoch  13 / 20  -  train_rmse: 0.9339699206323473
Epoch  14 / 20  -  train_rmse: 0.9297555277470417
Epoch  15 / 20  -  train_rmse: 0.925774277734708
Epoch  16 / 20  -  train_rmse: 0.9219927551471755
Epoch  17 / 20  -  train_rmse: 0.9183727012901429
Epoch  18 / 20  -  train_rmse: 0.9148889708916637
Epoch  19 / 20  -  train_rmse: 0.9115234310108142
Epoch  20 / 20  -  train_rmse: 0.9082546680961849


KernelMF(gamma=0.01, lr=0.001, n_epochs=20, reg=0.005)

In [12]:
%%time
# Update model with new users
matrix_fact.update_users(X_train_update, y_train_update, lr=0.001, n_epochs=20, verbose=1)
pred = matrix_fact.predict(X_test_update)
mse = mean_squared_error(y_test_update, pred)
rmse = mse ** 0.5

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 1.0398021283704904
Epoch  2 / 20  -  train_rmse: 1.020478662600199
Epoch  3 / 20  -  train_rmse: 1.005848177251611
Epoch  4 / 20  -  train_rmse: 0.9945873305997027
Epoch  5 / 20  -  train_rmse: 0.9856928072003546
Epoch  6 / 20  -  train_rmse: 0.9785971571937674
Epoch  7 / 20  -  train_rmse: 0.9727595973845106
Epoch  8 / 20  -  train_rmse: 0.9678621394682763
Epoch  9 / 20  -  train_rmse: 0.9636714055292357
Epoch  10 / 20  -  train_rmse: 0.9600292639524814
Epoch  11 / 20  -  train_rmse: 0.9568136162592095
Epoch  12 / 20  -  train_rmse: 0.9539222387749668
Epoch  13 / 20  -  train_rmse: 0.9512883248557428
Epoch  14 / 20  -  train_rmse: 0.9488661345855075
Epoch  15 / 20  -  train_rmse: 0.9466218373534171
Epoch  16 / 20  -  train_rmse: 0.9445286520016715
Epoch  17 / 20  -  train_rmse: 0.9425511207862295
Epoch  18 / 20  -  train_rmse: 0.9406812404664886
Epoch  19 / 20  -  train_rmse: 0.9389024829919745
Epoch  20 / 20  -  train_rmse: 0.9371985595897832

Test RMSE:

/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 30. 546. 385. ... 556. 469. 451.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)


## Sigmoid核（Sigmoid Kernel）

Sigmoid核使用sigmoid函数进行非线性变换，可以将评分映射到指定范围内。
公式：`K(u,i) = a + c · sigmoid(P_u · Q_i^T + bias_u + bias_i)`
- a: 最小评分
- c: 评分范围（max_rating - min_rating）
- sigmoid: 将线性组合映射到[0,1]，然后缩放到评分范围

In [13]:
%%time 
matrix_fact = KernelMF(n_epochs = 20, n_factors = 100, verbose = 1, lr = 0.01, reg = 0.005, kernel='sigmoid')
matrix_fact.fit(X_train, y_train)

pred = matrix_fact.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = mse ** 0.5

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 1.7254817435371432
Epoch  2 / 20  -  train_rmse: 1.700308561074648
Epoch  3 / 20  -  train_rmse: 1.662185043939576
Epoch  4 / 20  -  train_rmse: 1.6209990781563473
Epoch  5 / 20  -  train_rmse: 1.5756493084466494
Epoch  6 / 20  -  train_rmse: 1.5233836503824907
Epoch  7 / 20  -  train_rmse: 1.46567575401769
Epoch  8 / 20  -  train_rmse: 1.40932913011497
Epoch  9 / 20  -  train_rmse: 1.358343947496736
Epoch  10 / 20  -  train_rmse: 1.3133049382237956
Epoch  11 / 20  -  train_rmse: 1.2739097343043695
Epoch  12 / 20  -  train_rmse: 1.2392799984032787
Epoch  13 / 20  -  train_rmse: 1.2087226907333743
Epoch  14 / 20  -  train_rmse: 1.1814018507037192
Epoch  15 / 20  -  train_rmse: 1.1569242988100157
Epoch  16 / 20  -  train_rmse: 1.134889550964239
Epoch  17 / 20  -  train_rmse: 1.114907541428803
Epoch  18 / 20  -  train_rmse: 1.0966348228684075
Epoch  19 / 20  -  train_rmse: 1.0798328723523434
Epoch  20 / 20  -  train_rmse: 1.0642610155332082

Test RMSE: 1.1109

/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[136.  19. 572. ...  36. 790.  56.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)


## RBF核（Radial Basis Function Kernel）

RBF核（径向基函数核）基于用户特征和物品特征之间的欧氏距离。
公式：`K(u,i) = a + c · exp(-γ · ||P_u - Q_i||²)`
- γ: 核系数，控制相似度衰减速度
- 距离越近，相似度越高（评分越高）
- 这是一种非线性核，可以捕获更复杂的模式

In [14]:
%%time 
matrix_fact = KernelMF(n_epochs = 20, n_factors = 100, verbose = 1, lr = 0.5, reg = 0.005, kernel='rbf')
matrix_fact.fit(X_train, y_train)

pred = matrix_fact.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = mse ** 0.5

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 1.2630148054790906
Epoch  2 / 20  -  train_rmse: 1.1119593632318976
Epoch  3 / 20  -  train_rmse: 1.046309128081497
Epoch  4 / 20  -  train_rmse: 1.0038939232144437
Epoch  5 / 20  -  train_rmse: 0.9750366888185648
Epoch  6 / 20  -  train_rmse: 0.9530835408199735
Epoch  7 / 20  -  train_rmse: 0.9360555418161207
Epoch  8 / 20  -  train_rmse: 0.9205584952800061
Epoch  9 / 20  -  train_rmse: 0.9107018156995483
Epoch  10 / 20  -  train_rmse: 0.9043164927951252
Epoch  11 / 20  -  train_rmse: 0.899012425556555
Epoch  12 / 20  -  train_rmse: 0.894199480303159
Epoch  13 / 20  -  train_rmse: 0.8912633512714686
Epoch  14 / 20  -  train_rmse: 0.8877404484501655
Epoch  15 / 20  -  train_rmse: 0.8867681214133077
Epoch  16 / 20  -  train_rmse: 0.8856239041297626
Epoch  17 / 20  -  train_rmse: 0.8853318001759479
Epoch  18 / 20  -  train_rmse: 0.8822575504809821
Epoch  19 / 20  -  train_rmse: 0.8828802118234508
Epoch  20 / 20  -  train_rmse: 0.8800306510622041

Test RMSE: 

/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[532. 715. 416. ... 412. 439. 429.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)


# Scikit-learn兼容性

由于我们的模型继承自scikit-learn的BaseEstimator，可以与scikit-learn的工具（如GridSearchCV）无缝集成。
这允许我们使用网格搜索来自动寻找最佳超参数。

In [ ]:
from sklearn.model_selection import GridSearchCV, ParameterGrid

# ============================================
# 网格搜索：自动寻找最佳超参数
# ============================================
# 定义参数网格，包含所有要尝试的超参数组合
param_grid = {
    'kernel': ['linear', 'sigmoid', 'rbf'],  # 3种核函数
    'n_factors': [10, 20, 50],                # 3种潜在因子数量
    'n_epochs': [10, 20, 50],                 # 3种训练轮数
    'reg': [0, 0.005, 0.1]                    # 3种正则化系数
}
# 总共 3×3×3×3 = 81 种组合

# 创建网格搜索对象
# 参数说明：
# - KernelMF(verbose=0): 基础模型（不显示训练过程）
# - scoring='neg_root_mean_squared_error': 使用负RMSE作为评分（越大越好）
# - param_grid: 参数网格
# - n_jobs=-1: 使用所有CPU核心并行计算
# - cv=5: 5折交叉验证
# - verbose=1: 显示搜索进度
grid_search = GridSearchCV(KernelMF(verbose=0), scoring='neg_root_mean_squared_error', 
                          param_grid=param_grid, n_jobs=-1, cv=5, verbose=1)
grid_search.fit(X_train, y_train)

# 输出说明：
# - "Fitting 5 folds for each of 81 candidates, totalling 405 fits"
#   表示要对81种参数组合各进行5折交叉验证，总共405次训练
# - 这个过程可能需要较长时间（几分钟到几十分钟，取决于数据大小）
# - 最终会找到在交叉验证上表现最好的参数组合

Fitting 5 folds for each of 81 candidates, totalling 405 fits


/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[882.  62. 251. ... 235. 284. 508.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)
/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1041.  286.   30. ...  368.  107.  378.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)
/Users/shawnzhang/Menu/Recommedation_Project/matrix-factorization/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is 

GridSearchCV(cv=5, estimator=KernelMF(gamma=0.01, verbose=0), n_jobs=-1,
             param_grid={'kernel': ['linear', 'sigmoid', 'rbf'],
                         'n_epochs': [10, 20, 50], 'n_factors': [10, 20, 50],
                         'reg': [0, 0.005, 0.1]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [16]:
# ============================================
# 查看网格搜索结果
# ============================================
# best_score_: 最佳交叉验证分数（负RMSE，所以是负数）
# 实际RMSE = -best_score_
grid_search.best_score_

# best_params_: 最佳参数组合
# 这些参数在交叉验证上表现最好
grid_search.best_params_

# 输出说明：
# - best_score_ 是负的RMSE值（因为scoring='neg_root_mean_squared_error'）
#   实际最佳RMSE = -best_score_
# - best_params_ 包含最佳的超参数组合
#   例如：{'kernel': 'linear', 'n_factors': 50, 'n_epochs': 20, 'reg': 0.005}
# - 可以使用这些参数重新训练最终模型

np.float64(-0.9252502226068511)

{'kernel': 'linear', 'n_epochs': 50, 'n_factors': 50, 'reg': 0.1}